In [ ]:
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
assert gensim.models.doc2vec.FAST_VERSION > -1, "This will be painfully slow otherwise"
import multiprocessing
import os
import nltk
from nltk.tokenize import word_tokenize
import logging
from gensim.test.test_doc2vec import ConcatenatedDoc2Vec
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [ ]:
nltk.download('punkt')

In [ ]:
def read_data(base_path):
    reviews = []
    for temp,j,y in os.walk(base_path):
        if os.path.basename(os.path.normpath(temp)) in ['neg', 'pos', 'unsup']:
            for i, f in enumerate(os.listdir(temp)):
                if f.endswith('.txt'):
                    with open(os.path.join(temp, f), 'r') as file:
                        text = word_tokenize(file.read())
                        text = [x.lower() for x in text]
                        reviews.append(TaggedDocument(text, [i]))                
    return reviews

    def train_doc2vec(reviews, epochs, vector_size, window, dm, min_count = 2, dm_concat = 0, hs = 0, negative = 5):
        if dm == 0:
            model = Doc2Vec(vector_size=vector_size, min_count=min_count, epochs=epochs, dm=dm, dbow_words=1, hs=hs, negative=negative,
                        workers=multiprocessing.cpu_count())
        else:
            model = Doc2Vec(vector_size=vector_size, min_count=min_count, epochs=epochs, dm=dm, hs=hs, negative=negative,
                        workers=multiprocessing.cpu_count())

        model.build_vocab(reviews)
        model.train(reviews, total_examples=model.corpus_count, epochs=model.epochs)
        model.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)
        model.save("doc2vec/{}_{}_{}_{}_{}_{}_{}.model".format(epochs, vector_size, window, dm, 
                                                               min_count, hs, negative))
        print("Model Saved")

## Read Unsupervised Data

In [ ]:
data_path = 'datasets/aclImdb/'

In [ ]:
reviews = read_data(data_path)

In [ ]:
print(len(reviews))

## Train Doc2Vec

In [ ]:
train_doc2vec(reviews, epochs = 10, vector_size = 100, window = 5, dm = 0, min_count=5)

In [ ]:
train_doc2vec(reviews, epochs = 10, vector_size = 100, window = 5, dm = 0)

In [ ]:
train_doc2vec(reviews, epochs = 10, vector_size = 100, window = 5, dm = 1)

In [ ]:
train_doc2vec(reviews, epochs = 20, vector_size = 100, window = 10, dm = 1)

In [ ]:
train_doc2vec(reviews, epochs = 10, vector_size = 200, window = 10, dm = 1)

In [ ]:
train_doc2vec(reviews, epochs = 10, vector_size = 200, window = 10, dm=0)

In [ ]:
train_doc2vec(reviews, epochs = 10, vector_size = 100, window = 5, dm=1, dm_concat=1, hs=1)

In [ ]:
train_doc2vec(reviews, epochs = 10, vector_size = 100, min_count = 2, dm=1, window=15, negative=15, hs=0)

In [ ]:
epochs = 10
model2 = Doc2Vec(vector_size=200, min_count=2, epochs=epochs, dm=0, window = 10, dbow_words=1, hs=0, negative=5,workers=multiprocessing.cpu_count())        
model2.build_vocab(reviews)
model2.train(reviews, total_examples=model2.corpus_count, epochs=epochs)
model2.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)
model2.save("doc2vec/best_dbowmodel.model")

In [ ]:
epochs = 20
model1 = Doc2Vec(vector_size=100, min_count=2, epochs=epochs, dm=1, window = 10, workers=multiprocessing.cpu_count())        
model1.build_vocab(reviews)
model1.train(reviews, total_examples=model1.corpus_count, epochs=epochs)
model1.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)


doc2vec/10_200_10_0_2_0_5.model

In [ ]:
model1.save("doc2vec/dbow_dm_model1.model")
model2.save("doc2vec/dbow_dm_model2.model")



In [ ]:
from sklearn.model_selection import ParameterGrid


param_grid = {'epochs': [10, 20], 
              'vector_size': [100],
              'min_count': [2],
              'dm': [0, 1],
              'window': [5, 10],
              'hs': [0, 1],
              'negative': [5, 15]
             }
grid = ParameterGrid(param_grid)
for params in grid:
    print(params)
    train_doc2vec(documents, max_epochs = params['max_epochs'], vec_size = params['vec_size'], min_count = params['min_count'], dm = params['dm'])